# Chapter 1: Function Calling (without SK)
## [reminder] Classic Chat Completion 

In [1]:
import os  
from openai import AzureOpenAI  
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(override=True)

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY")  

# Initialize Azure OpenAI Service client with key-based authentication    
client = AzureOpenAI(  
    azure_endpoint=endpoint,  
    api_key=subscription_key,  
    api_version="2024-10-21",
)
    
#Prepare the chat prompt 
prompt = [
    {
        "role": "system",
        "content": [
            {
                "type": "text",
                "text": "You are a poetic AI assistant that helps people be creative."
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Write a haiku about Semantic Kernel."
            }
        ]
    }
] 
    
# Generate the completion  
completion = client.chat.completions.create(  
    model=deployment,
    messages=prompt,
    max_tokens=50,  
    temperature=1,  
    top_p=1,  
    stop=None,  
    stream=False
)

print("Generated Haiku:")
print(completion.choices[0].message.content)  

Generated Haiku:
Mind’s code intertwined,  
Semantic Kernel breathes life,  
Thoughts in harmony.


## Chat Completion with Tool Calling
### 1. Request Tool Call from GPT-4.1

In [2]:
client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-10-21",
 )

# Define a simple function schema
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "Get the current time in a specified location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city or timezone (e.g., 'Europe/Zurich')"
                    }
                },
                "required": ["location"]
            },
        }
    },
]

messages = [{"role": "user", "content": "What time is it in Tokyo?"}]

# Call chat completion with function calling
response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    messages=messages,
    tools=tools,
)

response_message = response.choices[0].message
messages.append(response_message)

print("Response with function call:")
print(response_message)

Response with function call:
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_hQrdg35ihxnWvW3hx3SXyOpS', function=Function(arguments='{"location":"Asia/Tokyo"}', name='get_current_time'), type='function')])


### 2. Execute Tool

In [ ]:
from datetime import datetime
import json
from zoneinfo import ZoneInfo

def get_current_time(location: str) -> str:
    """Return the current timestamp in the specified timezone as ISO-formatted string"""
    tz = ZoneInfo(location)
    now = datetime.now(tz)
    return now.strftime("%H:%M:%S")

# Execute the function call if OpenAI requests it
if response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            if tool_call.function.name == "get_current_time":    
                args = json.loads(tool_call.function.arguments)
                fn_result = get_current_time(location=args.get("location"))
                
                print(f"Tool call result:\n{fn_result}")
                
                messages.append({
                                "tool_call_id": tool_call.id,
                                "role": "tool",
                                "name": "get_current_time",
                                "content": fn_result,
                            })


Function call result:
13:44:53


### 3. Let GPT-4.1 process Tool result

In [ ]:
# Second API call: Get the final response from the model
final_response = client.chat.completions.create(
    model=deployment,
    messages=messages,
)

print(f"Final response after tool call:\n{final_response.choices[0].message.content}")

Final response after function call:
The current time in Tokyo is approximately 13:44 (1:44 PM).


# Chapter 2: Semantic Kernel
## [optional] Setup: Tracing

In [ ]:
import logging, sys
root = logging.getLogger()
handler = logging.StreamHandler(stream=sys.stdout)
handler.setLevel(logging.DEBUG)
root.handlers.clear()
root.addHandler(handler)
root.setLevel(logging.DEBUG)
logging.debug("Now you will see this in the notebook output")

In [60]:
import logging

from azure.monitor.opentelemetry.exporter import (
    AzureMonitorLogExporter,
    AzureMonitorMetricExporter,
    AzureMonitorTraceExporter,
)

from opentelemetry import trace
from opentelemetry._logs import set_logger_provider
from opentelemetry.metrics import set_meter_provider
from opentelemetry.sdk._logs import LoggerProvider, LoggingHandler
from opentelemetry.sdk._logs.export import BatchLogRecordProcessor
from opentelemetry.sdk.metrics import MeterProvider
from opentelemetry.sdk.metrics.export import PeriodicExportingMetricReader
from opentelemetry.sdk.metrics.view import DropAggregation, View
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.semconv.resource import ResourceAttributes
from opentelemetry.trace import set_tracer_provider

# Replace the connection string with your Application Insights connection string
connection_string = "InstrumentationKey=9abfbfb8-6951-416b-8c14-919c731e4af1;IngestionEndpoint=https://swedencentral-0.in.applicationinsights.azure.com/;LiveEndpoint=https://swedencentral.livediagnostics.monitor.azure.com/;ApplicationId=41515bba-845f-469b-8520-40e7900fcc61"

# Create a resource to represent the service/sample
resource = Resource.create({ResourceAttributes.SERVICE_NAME: "telemetry-application-insights-quickstart"})


def set_up_logging():
    exporter = AzureMonitorLogExporter(connection_string=connection_string)

    # Create and set a global logger provider for the application.
    logger_provider = LoggerProvider(resource=resource)
    # Log processors are initialized with an exporter which is responsible
    # for sending the telemetry data to a particular backend.
    logger_provider.add_log_record_processor(BatchLogRecordProcessor(exporter))
    # Sets the global default logger provider
    set_logger_provider(logger_provider)

    # Create a logging handler to write logging records, in OTLP format, to the exporter.
    handler = LoggingHandler()
    # Add filters to the handler to only process records from semantic_kernel.
    handler.addFilter(logging.Filter("semantic_kernel"))
    # Attach the handler to the root logger. `getLogger()` with no arguments returns the root logger.
    # Events from all child loggers will be processed by this handler.
    logger = logging.getLogger()
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)


def set_up_tracing():
    exporter = AzureMonitorTraceExporter(connection_string=connection_string)

    # Initialize a trace provider for the application. This is a factory for creating tracers.
    tracer_provider = TracerProvider(resource=resource)
    # Span processors are initialized with an exporter which is responsible
    # for sending the telemetry data to a particular backend.
    tracer_provider.add_span_processor(BatchSpanProcessor(exporter))
    # Sets the global default tracer provider
    set_tracer_provider(tracer_provider)


def set_up_metrics():
    exporter = AzureMonitorMetricExporter(connection_string=connection_string)

    # Initialize a metric provider for the application. This is a factory for creating meters.
    meter_provider = MeterProvider(
        metric_readers=[PeriodicExportingMetricReader(exporter, export_interval_millis=5000)],
        resource=resource,
        views=[
            # Dropping all instrument names except for those starting with "semantic_kernel"
            View(instrument_name="*", aggregation=DropAggregation()),
            View(instrument_name="semantic_kernel*"),
        ],
    )
    # Sets the global default meter provider
    set_meter_provider(meter_provider)


# This must be done before any other telemetry calls
set_up_logging()
set_up_tracing()
set_up_metrics()

## [optional] Setup: Load .env vars

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

# endpoint = "https://ai-serv-eval060765329650.openai.azure.com/models"
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
deployment = "gpt-4.1-mini"


## 2.1 Chat Completion with Semantic Kernel


### Introduction
[Introduction to Semantic Kernel Notebook](01-intro-to-semantic-kernel/01-intro.ipynb#Functions-and-Plugins-in-SK)


## 2.2 Agents in Semantic Kernel
### Chat Completion Agent without Tool Calling


In [ ]:
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

service_id = "sk-service"  # Optional service ID for targeting specific services

chat_completion_service = AzureChatCompletion()

tracer = trace.get_tracer(__name__) 
with tracer.start_as_current_span("agent-thread"):
    # Initialize a chat agent with basic instructions
    agent = ChatCompletionAgent(
        # service=inference_completion_service,
        service=chat_completion_service,
        name="SK-Assistant",
        instructions="You are a helpful assistant."
    )

    try:
        response = await agent.get_response(messages="Write a haiku about Semantic Kernel.")
        if response and response.content:
            print(response.content)
        else:
            print("No content received from the API.")
    except Exception as e:
        print(f"An error occurred: {e}")


NameError: name 'trace' is not defined

In [ ]:

tracer = trace.get_tracer(__name__) 
with tracer.start_as_current_span("agent-thread"):
    # Initialize a chat agent with basic instructions
    agent = ChatCompletionAgent(
        # service=inference_completion_service,
        service=chat_completion_service,
        name="SK-Assistant",
        instructions="You are a helpful assistant."
    )

    try:
        response = await agent.get_response(messages="Write a haiku about Semantic Kernel.")
        if response and response.content:
            print(response.content)
        else:
            print("No content received from the API.")
    except Exception as e:
        print(f"An error occurred: {e}")


### Chat Completion Agent with Tool Calling

In [109]:
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions import kernel_function

# Define a sample plugin with tools (functions) for the sample conversation
class MenuPlugin:
    """A sample Menu Plugin used for the concept sample."""

    @kernel_function(description="Provides a list of specials from the menu.")
    def get_specials(self, menu_item: str) -> Annotated[str, "Returns the specials from the menu."]:
        return """
        Special Soup: Clam Chowder
        Special Salad: Cobb Salad
        Special Drink: Chai Tea
        """ 

    @kernel_function(description="Provides the price of the requested menu item.")
    def get_item_price(
        self, menu_item: Annotated[str, "The name of the menu item."]
    ) -> Annotated[str, "Returns the price of the menu item."]:
        return "$9.99"

# Simulate a conversation with the agent
USER_INPUTS = [
    "Hello",
    "What is the special soup?",
    "What is the special drink?",
    "How much is it?",
    "Thank you",
]

# 1. Create a Semantic Kernel agent for the OpenAI Responses API
menu_agent = ChatCompletionAgent(
    service=chat_completion_service,
    instructions="Answer questions about the menu.",
    description="An agent that answers questions about the dinner menu.",
    name="DinnerMenuAgent",
    plugins=[MenuPlugin()],
    function_choice_behavior=FunctionChoiceBehavior.Auto()
)

# 2. Create a thread for the agent
# If no thread is provided, a new thread will be created and returned with the initial response
# Define the thread and tracer for the agent
thread = ChatHistoryAgentThread()
agent_id = menu_agent.id
print(f"Agent ID: {agent_id}")

tracer = trace.get_tracer(__name__)
with tracer.start_as_current_span(f"agent-{agent_id}") as span:
    try:
        for user_input in USER_INPUTS:
            print(f"\n# {AuthorRole.USER}: '{user_input}'")

            response = await menu_agent.get_response(
                messages=user_input,
                thread=thread,
            )
            
            print(f"# {response.name}: {response.content}")

            thread = response.thread
            span.set_attribute("gen_ai.thread.id", thread.id)
            span.set_attribute("thread.id", thread.id)

    finally:
        print(f"\nThread ID: {thread.id}")
        # await thread.delete() if thread else None


Agent ID: 41ad8a74-1801-4d46-bfb2-8270456e47b7

# AuthorRole.USER: 'Hello'
# DinnerMenuAgent: Hello! How can I assist you with the menu today?

# AuthorRole.USER: 'What is the special soup?'
# DinnerMenuAgent: Hello! How can I assist you with the menu today?

# AuthorRole.USER: 'What is the special soup?'
# DinnerMenuAgent: The special soup today is Clam Chowder. Would you like to know more about it or something else from the menu?

# AuthorRole.USER: 'What is the special drink?'
# DinnerMenuAgent: The special soup today is Clam Chowder. Would you like to know more about it or something else from the menu?

# AuthorRole.USER: 'What is the special drink?'
# DinnerMenuAgent: The special drink is Chai Tea. Would you like to know the price or details about it?

# AuthorRole.USER: 'How much is it?'
# DinnerMenuAgent: The special drink is Chai Tea. Would you like to know the price or details about it?

# AuthorRole.USER: 'How much is it?'
# DinnerMenuAgent: The price of the special drink, Ch

### Create Tool Call from OpenAPI
1. Method: provide a yaml file directly

In [36]:
openapi_library_yaml = """openapi: 3.0.3
info:
  title: Open Library Search API
  version: "1.0.0"
  description: >
    Search works and editions via Open Library’s Solr-backed endpoint.  
    Returns both work-level (author, first publish year, etc.) and edition-level info in one call :contentReference[oaicite:0]{index=0}
servers:
  - url: https://openlibrary.org
paths:
  /search.json:
    get:
      operationId: search_json_get
      summary: Search for works and editions
      parameters:
        - name: q
          in: query
          description: Full-text Solr query across all fields :contentReference[oaicite:1]{index=1}
          schema:
            type: string
        - name: title
          in: query
          description: Search by title :contentReference[oaicite:2]{index=2}
          schema:
            type: string
        - name: author
          in: query
          description: Search by author name :contentReference[oaicite:3]{index=3}
          schema:
            type: string
        - name: page
          in: query
          description: Page number (starts at 1) :contentReference[oaicite:4]{index=4}
          schema:
            type: integer
            minimum: 1
        - name: limit
          in: query
          description: Number of results per page :contentReference[oaicite:5]{index=5}
          schema:
            type: integer
            minimum: 1
        - name: sort
          in: query
          description: Sort order (e.g. `new`, `old`, `random`) :contentReference[oaicite:6]{index=6}
          schema:
            type: string
        - name: fields
          in: query
          description: Comma-separated list of fields to include (e.g. `*`, `title,author_name`) :contentReference[oaicite:7]{index=7}
          schema:
            type: string
        - name: lang
          in: query
          description: Preferred language (ISO 639-1 code) :contentReference[oaicite:8]{index=8}
          schema:
            type: string
      responses:
        "200":
          description: A page of search results
          content:
            application/json:
              schema:
                $ref: "#/components/schemas/SearchResponse"
components:
  schemas:
    SearchResponse:
      type: object
      properties:
        start:
          type: integer
          description: Index of first returned document :contentReference[oaicite:9]{index=9}
        num_found:
          type: integer
          description: Total number of matches :contentReference[oaicite:10]{index=10}
        numFoundExact:
          type: boolean
          description: Whether `num_found` is an exact count
        docs:
          type: array
          items:
            $ref: "#/components/schemas/Doc"
    Doc:
      type: object
      additionalProperties: true
      description: >
        A single result document with work- and edition-level fields; may include nested edition docs :contentReference[oaicite:11]{index=11}
      properties:
        key:
          type: string
          example: "/works/OL27448W"
        title:
          type: string
        author_name:
          type: array
          items:
            type: string
        author_key:
          type: array
          items:
            type: string
        first_publish_year:
          type: integer
        cover_i:
          type: integer
        has_fulltext:
          type: boolean
        edition_count:
          type: integer
        ia:
          type: array
          items:
            type: string
        public_scan_b:
          type: boolean
        editions:
          type: object
          description: Embedded edition‐level search results
          properties:
            numFound:
              type: integer
            start:
              type: integer
            numFoundExact:
              type: boolean
            docs:
              type: array
              items:
                type: object
                additionalProperties: true
                properties:
                  key:
                    type: string
                  title:
                    type: string
"""

import yaml
import json
openapi_library_dict = yaml.safe_load(openapi_library_yaml)

print(json.dumps(openapi_library_dict, indent=2))

{
  "openapi": "3.0.3",
  "info": {
    "title": "Open Library Search API",
    "version": "1.0.0",
    "description": "Search works and editions via Open Library\u2019s Solr-backed endpoint.   Returns both work-level (author, first publish year, etc.) and edition-level info in one call :contentReference[oaicite:0]{index=0}\n"
  },
  "servers": [
    {
      "url": "https://openlibrary.org"
    }
  ],
  "paths": {
    "/search.json": {
      "get": {
        "operationId": "search_json_get",
        "summary": "Search for works and editions",
        "parameters": [
          {
            "name": "q",
            "in": "query",
            "description": "Full-text Solr query across all fields :contentReference[oaicite:1]{index=1}",
            "schema": {
              "type": "string"
            }
          },
          {
            "name": "title",
            "in": "query",
            "description": "Search by title :contentReference[oaicite:2]{index=2}",
            "schema": 

In [ ]:
openapi_weather_yaml = """openapi: 3.0.0
info:
  title: Open-Meteo APIs
  description: 'Open-Meteo offers free weather forecast APIs for open-source developers and non-commercial use. No API key is required.'
  version: '1.0'
  contact:
    name: Open-Meteo
    url: https://open-meteo.com
    email: info@open-meteo.com
  license:
    name: Attribution 4.0 International (CC BY 4.0)
    url: https://creativecommons.org/licenses/by/4.0/
  termsOfService: https://open-meteo.com/en/features#terms
servers:
  - url: https://api.open-meteo.com
paths:
  /v1/forecast:
    get:
      operationId: get_weather_forecast
      tags:
      - Weather Forecast APIs
      summary: 7 day weather forecast for coordinates
      description: 7 day weather variables in hourly and daily resolution for given WGS84 latitude and longitude coordinates. Available worldwide.
      parameters:
      - name: hourly
        in: query
        explode: false
        schema:
          type: array
          items:
            type: string
            enum:
            - temperature_2m
            - relative_humidity_2m
            - dew_point_2m
            - apparent_temperature
            - pressure_msl
            - cloud_cover
            - cloud_cover_low
            - cloud_cover_mid
            - cloud_cover_high
            - wind_speed_10m
            - wind_speed_80m
            - wind_speed_120m
            - wind_speed_180m
            - wind_direction_10m
            - wind_direction_80m
            - wind_direction_120m
            - wind_direction_180m
            - wind_gusts_10m
            - shortwave_radiation
            - direct_radiation
            - direct_normal_irradiance
            - diffuse_radiation
            - vapour_pressure_deficit
            - evapotranspiration
            - precipitation
            - weather_code
            - snow_height
            - freezing_level_height
            - soil_temperature_0cm
            - soil_temperature_6cm
            - soil_temperature_18cm
            - soil_temperature_54cm
            - soil_moisture_0_1cm
            - soil_moisture_1_3cm
            - soil_moisture_3_9cm
            - soil_moisture_9_27cm
            - soil_moisture_27_81cm
      - name: daily
        in: query
        schema:
          type: array
          items:
            type: string
            enum:
            - temperature_2m_max
            - temperature_2m_min
            - apparent_temperature_max
            - apparent_temperature_min
            - precipitation_sum
            - precipitation_hours
            - weather_code
            - sunrise
            - sunset
            - wind_speed_10m_max
            - wind_gusts_10m_max
            - wind_direction_10m_dominant
            - shortwave_radiation_sum
            - uv_index_max
            - uv_index_clear_sky_max
            - et0_fao_evapotranspiration
      - name: latitude
        in: query
        required: true
        description: "WGS84 coordinate"
        schema:
          type: number
          format: double
      - name: longitude
        in: query
        required: true
        description: "WGS84 coordinate"
        schema:
          type: number
          format: double
      - name: current_weather
        in: query
        schema:
          type: boolean
      - name: temperature_unit
        in: query
        schema:
          type: string
          default: celsius
          enum:
          - celsius
          - fahrenheit
      - name: wind_speed_unit
        in: query
        schema:
          type: string
          default: kmh
          enum:
          - kmh
          - ms
          - mph
          - kn
      - name: timeformat
        in: query
        description: If format `unixtime` is selected, all time values are returned in UNIX epoch time in seconds. Please not that all time is then in GMT+0! For daily values with unix timestamp, please apply `utc_offset_seconds` again to get the correct date.
        schema:
          type: string
          default: iso8601
          enum:
          - iso8601
          - unixtime
      - name: timezone
        in: query
        description: If `timezone` is set, all timestamps are returned as local-time and data is returned starting at 0:00 local-time. Any time zone name from the [time zone database](https://en.wikipedia.org/wiki/List_of_tz_database_time_zones) is supported.
        schema:
          type: string
      - name: past_days
        in: query
        description: If `past_days` is set, yesterdays or the day before yesterdays data are also returned.
        schema:
          type: integer
          enum:
          - 1
          - 2
      responses:
        "200":
          description: OK
          content:
            application/json:
              schema:
                type: object
                properties:
                  latitude:
                    type: number
                    example: 52.52
                    description: WGS84 of the center of the weather grid-cell which was used to generate this forecast. This coordinate might be up to 5 km away.
                  longitude:
                    type: number
                    example: 13.419.52
                    description: WGS84 of the center of the weather grid-cell which was used to generate this forecast. This coordinate might be up to 5 km away.
                  elevation:
                    type: number
                    example: 44.812
                    description: The elevation in meters of the selected weather grid-cell. In mountain terrain it might differ from the location you would expect.
                  generationtime_ms:
                    type: number
                    example: 2.2119
                    description: Generation time of the weather forecast in milli seconds. This is mainly used for performance monitoring and improvements.
                  utc_offset_seconds:
                    type: integer
                    example: 3600
                    description: Applied timezone offset from the &timezone= parameter.
                  hourly:
                    $ref: "#/components/schemas/HourlyResponse"
                  hourly_units:
                    type: object
                    additionalProperties:
                      type: string
                    description: For each selected weather variable, the unit will be listed here.
                  daily:
                    $ref: "#/components/schemas/DailyResponse"
                  daily_units:
                    type: object
                    additionalProperties:
                      type: string
                    description: For each selected daily weather variable, the unit will be listed here.
                  current_weather:
                    $ref: "#/components/schemas/CurrentWeather"
        "400":
          description: Bad Request
          content:
            application/json:
              schema:
                type: object
                properties:
                  error:
                    type: boolean
                    description: Always set true for errors
                  reason:
                    type: string
                    description: Description of the error
                    example: "Latitude must be in range of -90 to 90°. Given: 300"
components:
  schemas:
    HourlyResponse:
      type: object
      description: For each selected weather variable, data will be returned as a floating point array. Additionally a `time` array will be returned with ISO8601 timestamps.
      required:
        - time
      properties:
        time:
          type: array
          items:
            type: string
        temperature_2m:
          type: array
          items:
            type: number
        relative_humidity_2m:
          type: array
          items:
            type: number
        dew_point_2m:
          type: array
          items:
            type: number
        apparent_temperature:
          type: array
          items:
            type: number
        pressure_msl:
          type: array
          items:
            type: number
        cloud_cover:
          type: array
          items:
            type: number
        cloud_cover_low:
          type: array
          items:
            type: number
        cloud_cover_mid:
          type: array
          items:
            type: number
        cloud_cover_high:
          type: array
          items:
            type: number
        wind_speed_10m:
          type: array
          items:
            type: number
        wind_speed_80m:
          type: array
          items:
            type: number
        wind_speed_120m:
          type: array
          items:
            type: number
        wind_speed_180m:
          type: array
          items:
            type: number
        wind_direction_10m:
          type: array
          items:
            type: number
        wind_direction_80m:
          type: array
          items:
            type: number
        wind_direction_120m:
          type: array
          items:
            type: number
        wind_direction_180m:
          type: array
          items:
            type: number
        wind_gusts_10m:
          type: array
          items:
            type: number
        shortwave_radiation:
          type: array
          items:
            type: number
        direct_radiation:
          type: array
          items:
            type: number
        direct_normal_irradiance:
          type: array
          items:
            type: number
        diffuse_radiation:
          type: array
          items:
            type: number
        vapour_pressure_deficit:
          type: array
          items:
            type: number
        evapotranspiration:
          type: array
          items:
            type: number
        precipitation:
          type: array
          items:
            type: number
        weather_code:
          type: array
          items:
            type: number
        snow_height:
          type: array
          items:
            type: number
        freezing_level_height:
          type: array
          items:
            type: number
        soil_temperature_0cm:
          type: array
          items:
            type: number
        soil_temperature_6cm:
          type: array
          items:
            type: number
        soil_temperature_18cm:
          type: array
          items:
            type: number
        soil_temperature_54cm:
          type: array
          items:
            type: number
        soil_moisture_0_1cm:
          type: array
          items:
            type: number
        soil_moisture_1_3cm:
          type: array
          items:
            type: number
        soil_moisture_3_9cm:
          type: array
          items:
            type: number
        soil_moisture_9_27cm:
          type: array
          items:
            type: number
        soil_moisture_27_81cm:
          type: array
          items:
            type: number
    DailyResponse:
      type: object
      description: For each selected daily weather variable, data will be returned as a floating point array. Additionally a `time` array will be returned with ISO8601 timestamps.
      properties:
        time:
          type: array
          items:
            type: string
        temperature_2m_max:
          type: array
          items:
            type: number
        temperature_2m_min:
          type: array
          items:
            type: number
        apparent_temperature_max:
          type: array
          items:
            type: number
        apparent_temperature_min:
          type: array
          items:
            type: number
        precipitation_sum:
          type: array
          items:
            type: number
        precipitation_hours:
          type: array
          items:
            type: number
        weather_code:
          type: array
          items:
            type: number
        sunrise:
          type: array
          items:
            type: number
        sunset:
          type: array
          items:
            type: number
        wind_speed_10m_max:
          type: array
          items:
            type: number
        wind_gusts_10m_max:
          type: array
          items:
            type: number
        wind_direction_10m_dominant:
          type: array
          items:
            type: number
        shortwave_radiation_sum:
          type: array
          items:
            type: number
        uv_index_max:
          type: array
          items:
            type: number
        uv_index_clear_sky_max:
          type: array
          items:
            type: number
        et0_fao_evapotranspiration:
          type: array
          items:
            type: number
      required:
        - time
    CurrentWeather:
      type: object
      description: "Current weather conditions with the attributes: time, temperature, wind_speed, wind_direction and weather_code"
      properties:
        time:
          type: string
        temperature:
          type: number
        wind_speed:
          type: number
        wind_direction:
          type: number
        weather_code:
          type: integer
      required:
        - time
        - temperature
        - wind_speed
        - wind_direction
        - weather_code
"""

import yaml
openapi_weather_dict = yaml.safe_load(openapi_weather_yaml)

2. Method: Fetch yaml from URL

In [3]:
import requests
import yaml, json

def fetch_openapi_spec(url: str) -> dict:
    """Fetch and parse OpenAPI spec from a given URL."""
    response = requests.get(url)
    response.raise_for_status()  # Raise an error for bad responses
    return yaml.safe_load(response.text)

# Fetch & parse the OpenAPI spec
## OpenLibrary Search API OpenAPI spec
url = "https://raw.githubusercontent.com/internetarchive/openlibrary-api/main/swagger.yaml"
openapi_library_dict = fetch_openapi_spec(url)

## OpenMeteo Weather API OpenAPI spec
url = "https://raw.githubusercontent.com/open-meteo/open-meteo/refs/heads/main/openapi.yml"
openapi_weather_dict = fetch_openapi_spec(url)

print(f"OpenAPI spec:\n{json.dumps(openapi_weather_dict, indent=2)}")

OpenAPI spec:
{
  "openapi": "3.0.0",
  "info": {
    "title": "Open-Meteo APIs",
    "description": "Open-Meteo offers free weather forecast APIs for open-source developers and non-commercial use. No API key is required.",
    "version": "1.0",
    "contact": {
      "name": "Open-Meteo",
      "url": "https://open-meteo.com",
      "email": "info@open-meteo.com"
    },
    "license": {
      "name": "Attribution 4.0 International (CC BY 4.0)",
      "url": "https://creativecommons.org/licenses/by/4.0/"
    },
    "termsOfService": "https://open-meteo.com/en/features#terms"
  },
  "paths": {
    "/v1/forecast": {
      "servers": [
        {
          "url": "https://api.open-meteo.com"
        }
      ],
      "get": {
        "tags": [
          "Weather Forecast APIs"
        ],
        "summary": "7 day weather forecast for coordinates",
        "description": "7 day weather variables in hourly and daily resolution for given WGS84 latitude and longitude coordinates. Available worl

### Create a Plugin from OpenAPI spec
1. OpenLibrary

In [37]:
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior

kernel = Kernel()
kernel.add_service(
    service=chat_completion_service,
)

book_search_plugin = kernel.add_plugin_from_openapi(
    openapi_parsed_spec=openapi_library_dict,
    plugin_name="OpenLibrarySearch",
    description="Access to an open library to find books, authors, titles.",
)

library_agent = ChatCompletionAgent(
    service=chat_completion_service,
    instructions="You are a helpful assistant that can search for books in the OpenLibrary. Whenever possible call the OpenLibrary Search API to find books, authors, titles.",
    description="This agent can search for books using the Open Library Search API.",
    name="BookSearchAgent",
    plugins=[book_search_plugin],
    function_choice_behavior=FunctionChoiceBehavior.Auto()
)

prompt = "List 10 books written by Isaac Asimov."
prompt = "Give me 5 books by James Hollis."

thread = ChatHistoryAgentThread()

response = await library_agent.get_response(
                messages=prompt,
                thread=thread,
                kernel=kernel,
            )
            
print(f"# {response.name}: {response.content}")

# BookSearchAgent: Here are 5 books by James Hollis:

1. The Middle Passage (1993)
2. Under Saturn's Shadow (1994)
3. Finding Meaning in the Second Half of Life (2005)
4. The Eden Project (1998)

Unfortunately, only 4 books by James Hollis were found in the results from the OpenLibrary. Let me know if you want me to look for more or anything else!


2. OpenMeteo

In [32]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior

kernel = Kernel()

chat_completion_service = AzureChatCompletion()
kernel.add_service(
    service=chat_completion_service,
)

weather_plugin = kernel.add_plugin_from_openapi(
    # openapi_document_path="https://raw.githubusercontent.com/open-meteo/open-meteo/refs/heads/main/openapi.yml",
    openapi_parsed_spec=openapi_weather_dict,
    plugin_name="OpenMeteo",
    description="Provides weather forecasts."
)

weather_agent = ChatCompletionAgent(
    service=chat_completion_service,
    instructions="You are a helpful assistant that can provide weather forecasts.",
    name="WeatherAgent",
    plugins=[weather_plugin],
    function_choice_behavior=FunctionChoiceBehavior.Auto()
)

prompt = "Give me the forecast for Munich for the next 3 days?"
prompt = "What is the weather in Munich today?"

thread = ChatHistoryAgentThread()

response = await weather_agent.get_response(
                messages=prompt,
                thread=thread,
                kernel=kernel,
            )
            
print(f"# {response.name}: {response.content}")

# WeatherAgent: The current weather in Munich is 23.1°C with a light wind speed of 5.9 km/h coming from the west-northwest. The sky is partly cloudy.


In [38]:
# List all functions available in the agent

functions = []
for plugin in library_agent.kernel.plugins:
            functions_metadata = library_agent.kernel.plugins[plugin].get_functions_metadata()
            for function in functions_metadata:
                # Serialize metadata to a dictionary
                function_dict = function.model_dump()
                functions.append(function_dict)

print(f"Functions available in the agent: {functions}")

Functions available in the agent: [{'name': 'search_json_get', 'plugin_name': 'OpenLibrarySearch', 'description': 'Search for works and editions', 'parameters': [{'name': 'q', 'description': 'Full-text Solr query across all fields :contentReference[oaicite:1]{index=1}', 'default_value': '', 'type_': 'string', 'is_required': False, 'schema_data': {'type': 'string'}, 'include_in_function_choices': True}, {'name': 'title', 'description': 'Search by title :contentReference[oaicite:2]{index=2}', 'default_value': '', 'type_': 'string', 'is_required': False, 'schema_data': {'type': 'string'}, 'include_in_function_choices': True}, {'name': 'author', 'description': 'Search by author name :contentReference[oaicite:3]{index=3}', 'default_value': '', 'type_': 'string', 'is_required': False, 'schema_data': {'type': 'string'}, 'include_in_function_choices': True}, {'name': 'page', 'description': 'Page number (starts at 1) :contentReference[oaicite:4]{index=4}', 'default_value': '', 'type_': 'integer

In [19]:
# Print the raw response from Semantic Kernel Agent

print(f"\nThread ID: {thread.id}")
for message in thread._chat_history.messages:
    print(f"{message.role}: {message.content}")
    print(f">{message.items}")



Thread ID: thread_12eb4bb85e9848c1885c3db504252c25
AuthorRole.USER: What is the forecast for Munich, Germany?
>[TextContent(inner_content=None, ai_model_id=None, metadata={}, content_type='text', text='What is the forecast for Munich, Germany?', encoding=None)]
AuthorRole.ASSISTANT: 
>[FunctionCallContent(inner_content=None, ai_model_id=None, metadata={}, content_type='function_call', id='call_KJr21V7Np7kBYmSMOkHusx2g', call_id=None, index=None, name='OpenMeteo-get_weather_forecast', function_name='get_weather_forecast', plugin_name='OpenMeteo', arguments='{"latitude":48.1351,"longitude":11.582,"daily":["temperature_2m_max","temperature_2m_min","precipitation_sum","weather_code","wind_speed_10m_max"],"timezone":"Europe/Berlin"}')]
AuthorRole.TOOL: 
>[FunctionResultContent(inner_content=FunctionResult(function=KernelFunctionMetadata(name='get_weather_forecast', plugin_name='OpenMeteo', description='7 day weather forecast for coordinates', parameters=[KernelParameterMetadata(name='hourl

In [39]:
# Print all agentic steps in the thread
from semantic_kernel.contents import AuthorRole

print("\nMessages in the thread:")
for message in thread._chat_history.messages:
    print(f"{message.role}: {message.content}") 
    if message.role == AuthorRole.ASSISTANT:
        if message.items:
            print(f"Items: {message.items}")


Messages in the thread:
AuthorRole.USER: Give me 5 books by James Hollis.
AuthorRole.ASSISTANT: 
Items: [FunctionCallContent(inner_content=None, ai_model_id=None, metadata={}, content_type='function_call', id='call_UwNxd7ROY6v3KIZ7LugeyboS', call_id=None, index=None, name='OpenLibrarySearch-search_json_get', function_name='search_json_get', plugin_name='OpenLibrarySearch', arguments='{"author":"James Hollis","limit":5}')]
AuthorRole.TOOL: 
AuthorRole.ASSISTANT: Here are 5 books by James Hollis:

1. The Middle Passage (1993)
2. Under Saturn's Shadow (1994)
3. Finding Meaning in the Second Half of Life (2005)
4. The Eden Project (1998)

Unfortunately, only 4 books by James Hollis were found in the results from the OpenLibrary. Let me know if you want me to look for more or anything else!
Items: [TextContent(inner_content=None, ai_model_id=None, metadata={}, content_type='text', text="Here are 5 books by James Hollis:\n\n1. The Middle Passage (1993)\n2. Under Saturn's Shadow (1994)\n3. F

In [167]:
from semantic_kernel.agents import GroupChatOrchestration, RoundRobinGroupChatManager
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.contents import ChatMessageContent

agents = [library_agent, menu_agent]

def agent_response_callback(message: ChatMessageContent) -> None:
    print(f"**{message.name}**\n{message.content}")

orchestrator = ChatCompletionAgent(
    service=chat_completion_service,
    instructions="You are an orchestration agent that coordinates multiple agents.",
    description="This agent orchestrates multiple agents to answer questions.",
    name="Orchestrator-Agent",
    plugins=agents
)

task="what's the special soup on the menu tonight?"
task="Who wrote the 'The Great Gatsby'?"

thread = ChatHistoryAgentThread()

response = await orchestrator.get_response(
    messages=task,
    thread=thread,  # No thread provided, a new one will be created
    )

print(f"# {response.name}: {response.content}")


# Orchestrator-Agent: The novel "The Great Gatsby" was written by F. Scott Fitzgerald.


### [optional] See Agent Tool Definitions

In [ ]:
import json

agent = weather_agent

print("# Semantic Kernel was provided with the following tools:")

tools = []
for plugin in agent.kernel.plugins:
    functions_metadata = agent.kernel.plugins[plugin].get_functions_metadata()
    for function in functions_metadata:
        # Serialize metadata to a dictionary
        function_dict = function.model_dump()
        # function_dict["type"] = "tool_call"
        tools.append(function_dict)
print(f"## tool_definitions :\n {json.dumps(tools, indent=2)}")

### [optional] See Agent Chat History (including Tool Calls)

In [ ]:
chat_history = thread.get_messages()

print(f"# Chat history for thread {thread.id}:")    
print(f"## Chat History :\n {json.dumps([message.model_dump() async for message in chat_history], indent=2)}")


# Chat history for thread thread_ba1af91487344bbea517f7e10e379784:
## Chat History :
 [
  {
    "ai_model_id": null,
    "metadata": {},
    "content_type": "message",
    "role": "user",
    "name": null,
    "items": [
      {
        "ai_model_id": null,
        "metadata": {},
        "content_type": "text",
        "text": "Hello",
        "encoding": null
      }
    ],
    "encoding": null,
    "finish_reason": null,
    "status": null
  },
  {
    "ai_model_id": "gpt-4.1-nano",
    "metadata": {
      "logprobs": null,
      "id": "chatcmpl-BbuKQPwGE0Zp7Gdgo9lN31FRUOWTh",
      "created": 1748374362,
      "system_fingerprint": "fp_68472df8fd",
      "usage": {
        "prompt_tokens": 94,
        "completion_tokens": 16
      }
    },
    "content_type": "message",
    "role": "assistant",
    "name": "Host",
    "items": [
      {
        "ai_model_id": null,
        "metadata": {},
        "content_type": "text",
        "text": "Hello! Welcome to our menu. How can I assist 

# Chapter 3: Evaluations

In [ ]:
from utils.evaluation import AgentEvaluation

evaluator_config = {'tool_call_accuracy': True,
                    'intent_resolution': True,
                    'task_adherence': True
                   }

# Set the judge model and create an Agent Evaluation
judge_model_name="gpt-4.1"
agent_evaluator = AgentEvaluation(judge_model_name=judge_model_name)

# Run the evaluation
evaluation_result = agent_evaluator.evaluate(
    agent=agent,
    thread=thread,
    evaluator_config=evaluator_config
)

In [ ]:
metrics = evaluation_result.get("metrics")
print(f"Evaluation Metrics :\n {json.dumps(metrics, indent=2)}")

## Evaluation Metrics :
 {
  "intent_resolution.intent_resolution": 5.0,
  "intent_resolution.intent_resolution_threshold": 3.0,
  "task_adherence.task_adherence": 4.5,
  "task_adherence.task_adherence_threshold": 3.0
}


In [13]:
# Print the AI Foundry URL if available
studio_url = evaluation_result.get("studio_url")
print(studio_url)

https://ai.azure.com/build/evaluation/9f0398b5-5451-4856-ab91-c583af7a312f?wsid=/subscriptions/02ba4597-4783-4faf-aaa3-00814c82b7b9/resourceGroups/rg-ai-eval/providers/Microsoft.MachineLearningServices/workspaces/ai-proj-eval


### Load Synthetic Conversations (template)

In [ ]:
import pandas as pd
import json

filename = './data/conversations_20250426-0737_with_evals.parquet'
df = pd.read_parquet(filename)
df['query']=df['query'].apply(lambda x: json.loads(x))
df['response']=df['response'].apply(lambda x: json.loads(x))
df['tool_definitions'] = df['tool_definitions'].apply(lambda x: x['tool_definitions'])
df